In [24]:
host = "10.41.105.182"
port = 830
user = "admin_user"
password = "Ericsson1234"
ROW_LENGTH = 160

import traceback
import xml.dom.minidom
from ncclient import manager
from ncclient.xml_ import to_ele

def print_formatted(string):
    print("printing formatted str: " + str(string))
    try:
        print(xml.dom.minidom.parseString(string).toprettyxml())
    except Exception:
        print("Exception while printing xml dom")
        traceback.print_exc()
        print("Trying to print simplified version")
        print_simplified(string)


def print_simplified(string):
    row = ""
    index = 0
    if len(string) < ROW_LENGTH:
        print(string)
    else:
        for ch in string:
            if index < ROW_LENGTH:
                row += (ch)
                index += 1
            else:
                print(row)
                index = 0
                row = ""

m = manager.connect(host=host, port=port, username=user, password=password, hostkey_verify=False)

In [25]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:filter type="subtree">
    <if:interfaces xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces"/>
  </nc:filter>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

TimeoutExpiredError: ncclient timed out while waiting for an rpc reply.

### Here first try to recall the difference between the get and get-config operations. Q: Which datastore(s) are accessed in these operations?
from netconf rfc:

```txt
The <get-config> operation retrieves
   configuration data only, while the <get> operation retrieves
   configuration and state data.
```

```txt
     +-------------+                 +-----------+
     | <candidate> |                 | <startup> |
     |  (ct, rw)   |<---+       +--->| (ct, rw)  |
     +-------------+    |       |    +-----------+
            |           |       |           |
            |         +-----------+         |
            +-------->| <running> |<--------+
                      | (ct, rw)  |
                      +-----------+
                            |
                            |        // configuration transformations,
                            |        // e.g., removal of nodes marked as
                            |        // "inactive", expansion of
                            |        // templates
                            v
                      +------------+
                      | <intended> | // subject to validation
                      | (ct, ro)   |
                      +------------+
                            |        // changes applied, subject to
                            |        // local factors, e.g., missing
                            |        // resources, delays
                            |
       dynamic              |   +-------- learned configuration
       configuration        |   +-------- system configuration
       datastores -----+    |   +-------- default configuration
                       |    |   |
                       v    v   v
                    +---------------+
                    | <operational> | <-- system state
                    | (ct + cf, ro) |
                    +---------------+

     ct = config true; cf = config false
     rw = read-write; ro = read-only
     boxes denote named datastores
```

### Q: Identify at least one field in the response which is config-false (see the Yang model for help). Would this field appear in the response, if we performed a get-config operation instead of the get?

```yang
          container bridge-vlan {
            when "not(derived-from-or-self(../../bridge-type, 'two-port-mac-relay-bridge'))" {
              description
                "Applies to non TPMRs.";
            }
            reference
              "12.10 of IEEE Std 802.1Q-2022";

            leaf version {
              type uint16;
              config false;
              description
                "The version number supported.";
              reference
                "12.10.1.3 of IEEE Std 802.1Q-2022";

            }
```
```txt
7.1.  <get-config>

   Description:  Retrieve all or part of a specified configuration
      datastore.

   Parameters:
```
```txt
7.7.  <get>

   Description:  Retrieve running configuration and device state
      information.

   Parameters:
```



In [15]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces" xmlns:ianaift="urn:ietf:params:xml:ns:yang:iana-if-type">
    <nc:filter type="xpath" select="/if:interfaces/if:interface[if:type=derived-from-or-self(if:type, 'ianaift:ethernetCsmacd')]/enabled"/>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:2cbaf50b-59f8-4e1c-9c69-e01906c22fd8"><data><interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"><interface><name>LAN 1/0/4</name><enabled>true</enabled></interface><interface><name>LAN 1/0/5</name><enabled>true</enabled></interface><interface><name>LAN 1/0/7</name><enabled>true</enabled></interface><interface><name>LAN 1/0/8</name><enabled>true</enabled></interface><interface><name>LAN-DCN 1/0/0</name><enabled>true</enabled></interface><interface><name>WAN 1/1/1</name><enabled>true</enabled></interface></interfaces></data></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:2cbaf50b-59f8-4e1c-9c69-e01906c22fd8">
	<data>
		<interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces">
			<interface>
				<name>LAN 1/0/4</name>
				<enabled>true</enabled>
			</interface>
			<interface>
				<name>LAN 1/0/5</name

In [16]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0">
  <nc:filter type="subtree">
    <if:interfaces xmlns:if="urn:ietf:params:xml:ns:yang:ietf-interfaces">
      <if:interface>
        <if:enabled/>
        <if:link-up-down-trap-enable/>
      </if:interface>
    </if:interfaces>
  </nc:filter>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:bd591a7c-1d4a-41cc-9d96-5a0ec94a96e7"><data><interfaces xmlns="urn:ietf:params:xml:ns:yang:ietf-interfaces"><interface><name>CT 1/1/1</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/4</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/5</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/7</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN 1/0/8</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>LAN-DCN 1/0/0</name><enabled>true</enabled><link-up-down-trap-enable>enabled</link-up-down-trap-enable></interface><interface><name>

In [17]:
message = '''
<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:bp="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
    <nc:filter type="xpath" select="bp:bridges"/>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:d9f833a5-68fb-4318-8f24-5f16bf67d21a"><data><bridges xmlns="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge"><bridge><name>BRIDGE</name><address>98-A4-04-0C-33-F9</address><bridge-type>customer-vlan-bridge</bridge-type><components>1</components><component><name>DEFAULT</name><id>1</id><type>c-vlan-component</type><address>98-A4-04-0C-33-F9</address><traffic-class-enabled>true</traffic-class-enabled><filtering-database><aging-time>300</aging-time><vlan-registration-entry><database-id>1</database-id><vids>1</vids><entry-type>static</entry-type><port-map><port-ref>2</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin-control><vlan-transmitted>untagged</vlan-transmitted></static-vlan-registration-entries></port-map><port-map><port-ref>4</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin

In [18]:
message = '''
<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:bp="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
    <nc:filter type="xpath" select="bp:bridges/bp:bridge[name='BRIDGE']"/>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:8a3919f3-0bc1-4694-8276-5fb40050a90d"><data><bridges xmlns="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge"><bridge><name>BRIDGE</name><address>98-A4-04-0C-33-F9</address><bridge-type>customer-vlan-bridge</bridge-type><components>1</components><component><name>DEFAULT</name><id>1</id><type>c-vlan-component</type><address>98-A4-04-0C-33-F9</address><traffic-class-enabled>true</traffic-class-enabled><filtering-database><aging-time>300</aging-time><vlan-registration-entry><database-id>1</database-id><vids>1</vids><entry-type>static</entry-type><port-map><port-ref>2</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin-control><vlan-transmitted>untagged</vlan-transmitted></static-vlan-registration-entries></port-map><port-map><port-ref>4</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin

In [19]:
message = '''<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:bp="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
    <nc:filter type="xpath" select="bp:bridges/bp:bridge[name='BRIDGE']/bp:component[name='POTATO']"/>
</nc:get> '''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:4c190cab-370a-4260-897f-efd1445c6c98"><data/></rpc-reply>
<?xml version="1.0" ?>
<rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:4c190cab-370a-4260-897f-efd1445c6c98">
	<data/>
</rpc-reply>



In [ ]:
message = '''
<nc:get xmlns:nc="urn:ietf:params:xml:ns:netconf:base:1.0" xmlns:bp="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge">
    <nc:filter type="xpath" select="bp:bridges/bp:bridge[name='BRIDGE']"/>
</nc:get>'''
response = m.dispatch(to_ele(message))
print_formatted(response.xml)

printing formatted str: <rpc-reply xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:8a3919f3-0bc1-4694-8276-5fb40050a90d"><data><bridges xmlns="urn:ieee:std:802.1Q:yang:ieee802-dot1q-bridge"><bridge><name>BRIDGE</name><address>98-A4-04-0C-33-F9</address><bridge-type>customer-vlan-bridge</bridge-type><components>1</components><component><name>DEFAULT</name><id>1</id><type>c-vlan-component</type><address>98-A4-04-0C-33-F9</address><traffic-class-enabled>true</traffic-class-enabled><filtering-database><aging-time>300</aging-time><vlan-registration-entry><database-id>1</database-id><vids>1</vids><entry-type>static</entry-type><port-map><port-ref>2</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin-control><vlan-transmitted>untagged</vlan-transmitted></static-vlan-registration-entries></port-map><port-map><port-ref>4</port-ref><static-vlan-registration-entries><registrar-admin-control>fixed-new-ignored</registrar-admin